In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

In [14]:
df

,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,temp,dewp,humid,wind_dir,wind_speed,wind_gust,precip,pressure,visib,type,manufacturer,model,engines,seats,engine,arr_delay
0,B6,1201,N643JB,JFK,FLL,135.0,1069,21,50,73.04,66.92,81.14,160.0,13.80936,19.56326,0.0,1019.4,10.0,Fixed wing multi engine,AIRBUS,A320-232,2.0,200.0,Turbo-fan,1.0
1,DL,1773,N3743H,JFK,LAS,287.0,2248,15,35,71.06,42.98,36.28,230.0,14.96014,24.16638,0.0,1020.4,10.0,Fixed wing multi engine,BOEING,737-832,2.0,189.0,Turbo-jet,0.0
2,EV,4572,N11107,EWR,GSP,84.0,594,15,9,78.80,69.80,73.95,200.0,8.05546,24.16638,0.0,1017.5,5.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2.0,55.0,Turbo-fan,1.0
3,B6,179,N526JB,JFK,PHX,305.0,2153,17,35,44.06,14.00,29.23,270.0,17.26170,24.16638,0.0,1009.2,10.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,US,1733,N162UW,LGA,CLT,78.0,544,7,53,69.98,53.96,56.78,230.0,6.90468,24.16638,0.0,1021.3,10.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A321-211,2.0,199.0,Turbo-jet,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,UA,535,N525UA,JFK,LAX,323.0,2475,18,35,48.92,15.98,26.52,360.0,8.05546,24.16638,0.0,1025.3,10.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
9996,DL,1147,N977DL,LGA,ATL,106.0,762,20,0,75.02,57.92,55.28,190.0,13.80936,24.16638,0.0,1017.3,10.0,Fixed wing multi engine,MCDONNELL DOUGLAS AIRCRAFT CO,MD-88,2.0,142.0,Turbo-fan,1.0
9997,EV,4204,N12163,EWR,OKC,130.0,1325,20,9,37.04,33.98,88.57,240.0,9.20624,24.16638,0.0,1017.0,9.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2.0,55.0,Turbo-fan,0.0
9998,B6,431,N585JB,LGA,SRQ,146.0,1047,13,30,35.96,19.94,51.78,280.0,9.20624,18.41248,0.0,1016.9,10.0,Fixed wing multi engine,AIRBUS,A320-232,2.0,200.0,Turbo-fan,1.0


In [ ]:
df=pd.read_csv('/content/flight-data.csv')
df['arr_delay'].fillna(df['arr_delay'].median(), inplace=True)
df_1=df[['air_time','distance', 'hour', 'minute', 'temp', 'dewp', 'humid', 'wind_dir','wind_speed', 'wind_gust', 'precip', 'pressure', 'visib']]
df_1.fillna(df_1.median(), inplace=True)
df[['air_time','distance', 'hour', 'minute', 'temp', 'dewp', 'humid', 'wind_dir','wind_speed', 'wind_gust', 'precip', 'pressure', 'visib']]=df_1

### **After assessing the dataframe, I thought to omit some columns like 'carrier', 'flight', 'tailnum' 'type','manufacturer', 'model', 'engines', 'seats', 'engine' since arrival delay doesnt depend on them.It mostly depends on weather, distance and time.So, I chose to omit those columns.**

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 25 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   carrier       10000 non-null  object 
 1   flight        10000 non-null  int64  
 2   tailnum       9926 non-null   object 
 3   origin        10000 non-null  object 
 4   dest          10000 non-null  object 
 5   air_time      10000 non-null  float64
 6   distance      10000 non-null  int64  
 7   hour          10000 non-null  int64  
 8   minute        10000 non-null  int64  
 9   temp          10000 non-null  float64
 10  dewp          10000 non-null  float64
 11  humid         10000 non-null  float64
 12  wind_dir      10000 non-null  float64
 13  wind_speed    10000 non-null  float64
 14  wind_gust     10000 non-null  float64
 15  precip        10000 non-null  float64
 16  pressure      10000 non-null  float64
 17  visib         10000 non-null  float64
 18  type          8499 non-null

### **The dataset has 10000 observations, if we drop null values, 80% of the data is lost.So, I chose replacing the nulls with median value for preprocessing.**

## Splitting Data 

In [4]:
y=df['arr_delay']
X=df[['air_time','distance', 'hour', 'minute', 'temp', 'dewp', 'humid', 'wind_dir',
       'wind_speed', 'wind_gust', 'precip', 'pressure', 'visib']]

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
print(f'Training examples: {X_train.shape[0]:,}')
print(f'Test examples: {X_test.shape[0]:,}')

Training examples: 8,000
Test examples: 2,000


### Train_test_split is 0.2 i.e., 8000 Training samples and 2000 test samples.


## Logistic Regression Model- Grid Search CV

1. **Created a pipeline for logistic regression model with Ridge regression as penalty**
2. **Used 0.001, 0.1,1,10 as penalty strengths for grid search.** 

In [11]:
clf1 = LogisticRegression(penalty='l2', C=0.001, solver='lbfgs', random_state=0)
pipe1 = Pipeline([('scaler', StandardScaler()),('logreg', clf1)])
param1 = {'logreg__C':[0.001,0.1, 1, 10]}

lr_gs = GridSearchCV(estimator=pipe1, param_grid=param1,cv=10,scoring='roc_auc', refit=True)
lr_gs = lr_gs.fit(X_train, y_train)
lr_gs_score = lr_gs.score(X_test, y_test)
print(f'ROC_AUC for Logistic Regression Classifier  is {lr_gs_score:.2%}')

ROC_AUC for Logistic Regression Classifier  is 68.49%


### **Here,  the scoring metric is Area under curve**
### **For Logistic Regression,ROC AUC is 68.49%,which cannot be considered a good discrimination, not much better than a coin toss.**

## Decision Tree Classifier- Grid Search CV

1. **Created a pipeline for logistic regression model with max_depth=1 and 'entropy' as criterion.**
2. **Used 1,2,3,4 as max_depths for grid search.**

In [12]:
clf2 = DecisionTreeClassifier(max_depth=1, criterion='entropy', random_state=1)
pipe2 = Pipeline([('scaler', StandardScaler()),('tree', clf2)])
param2 = {'tree__max_depth': [1,2,3,4]}

dt_gs = GridSearchCV(estimator=pipe2, param_grid=param2,cv=10, scoring='roc_auc', refit=True)
dt_gs = dt_gs.fit(X_train, y_train)
dt_gs_score = dt_gs.score(X_test, y_test)
print(f' ROC_AUC for Decision Tree Classifier is {dt_gs_score:.2%}')

 ROC_AUC for Decision Tree Classifier is 61.92%


### **Here the scoring metric is Area under curve as well**
#### **ROC AUC is 61.92%, way lesser than that for logistic regression.But, this can be considered as poor discrimination**

## Support Vector Classifier - GridSearch CV

1. **Created a pipeline for SVM model with 'linear' kernel since the predicrtion id linear separable.**
2. **Since it is linear seperable, probability parameter is set True**
3. **Used 0.001, 0.1,1,10 aspenalty strengths for grid search.**

In [13]:
clf3 = SVC(kernel='linear',probability=True)
pipe3 = Pipeline([('scaler', StandardScaler()),('svc', clf3)])
param3 = {'svc__C':[0.001, 0.1, 1, 10]}

svc_gs = GridSearchCV(estimator=pipe3, param_grid=param3, scoring='roc_auc', refit=True)
svc_gs = svc_gs.fit(X_train, y_train)
svc_gs_score = svc_gs.score(X_test, y_test)
print(f'ROC_AUC for Support Vector Classifier is {svc_gs_score:.2%}')

ROC_AUC for Support Vector Classifier is 69.03%


#### **Here the scoring metric is ROC_AUC**
#### **ROC AUC is 69.03,which is better compared to logistic and decision tree classifiers**
#### **It took me aroung 7 minutes on a gpu to run this classifier, might be because of following reasons,**
##### 1.C parameter - greater the missclassification penalty, slower the process
##### 2.kernel - more complicated the kernel, slower the process (rbf is the most complex from the predefined ones)
##### 3.data size/dimensionality

## Ensemble of LR, DTC, SVC (Voting Classifier)

In [15]:
from sklearn.ensemble import VotingClassifier

In [16]:
params = {'LR__logreg__C':[0.001, 0.1, 1, 10], 
          'DTC__tree__max_depth': [1,2,3], 
          'SVC__svc__C': [1,2,3]
         }
ems = [('LR', pipe1),('DTC', pipe2),('SVC', pipe3)]
clf4 = VotingClassifier(estimators= ems, weights=None, voting='soft')
vc_gs = GridSearchCV(estimator=clf4, param_grid=params, scoring='roc_auc', refit=True)

vc_gs = vc_gs.fit(X_train, y_train)
vc_gs_score = vc_gs.score(X_test, y_test)

print(f'VotingClassifier Test ROC AUC: {vc_gs_score:.2%}')

KeyboardInterrupt: ignored

## **I tried to run this, waited for almost 2 hours.No error showed up.But I did not get any output.**

## Adaboost using Decison Tree Classifer

In [51]:
from sklearn.ensemble import AdaBoostClassifier
# from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

In [54]:
ada = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy', random_state=1, max_depth=1))
ada = ada.fit(X,y)
ada_roc_score=roc_auc_score(y, ada.predict_proba(X)[:, 1])
print(f'ROC_AUC for Adaboost  is {ada_roc_score:.2%}')

ROC_AUC for Logistic Regression Classifier  is 70.81%


### Ababoost classifier using Decision Tree Classifier with max_depth = 1 as base estimator
### ROC is 70.81% which is comparatively higher than all the above classifiers.ROC of 70% can be considered good but can be improved.